In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch
from models import nin_halfadd
import data
import util_fuc as util

def load_pretrained(filePath):
    model = nin_halfadd.Net()
    pretrained_model = torch.load(filePath)
    useState_dict = model.state_dict()
    preState_dict = pretrained_model['state_dict']
    useKeys = useState_dict.keys()
    preKeys = preState_dict.keys()
    j = 0
    print (len(useKeys))
    print (len(preKeys))
    for key in useKeys:
        if j == 50:
            j = 0
        if key.find('num_batches_tracked') == -1:
            useState_dict[key].data = preState_dict[preKeys[j]].data
            print (key , preKeys[j])
            j +=1
    model.load_state_dict(useState_dict)
    model.to(device)
    return model

In [2]:
testset = data.dataset(root='./data/', train=False)
testloader = torch.utils.data.DataLoader(testset,
                             batch_size=512, shuffle=False, num_workers=4)
device = torch.device('cuda:1' if torch.cuda.is_available() else "cpu")
model = load_pretrained('nin.best.pth.tar')
bin_op = util.BinOp(model)
bin_op.binarization()
model = model.eval()

64
50
conv1.weight xnor.0.weight
conv1.bias xnor.0.bias
bn1.running_mean xnor.1.running_mean
bn1.running_var xnor.1.running_var
bconv2.bn.weight xnor.3.bn.weight
bconv2.bn.bias xnor.3.bn.bias
bconv2.bn.running_mean xnor.3.bn.running_mean
bconv2.bn.running_var xnor.3.bn.running_var
bconv2.conv.weight xnor.3.conv.weight
bconv2.conv.bias xnor.3.conv.bias
bconv3.bn.weight xnor.4.bn.weight
bconv3.bn.bias xnor.4.bn.bias
bconv3.bn.running_mean xnor.4.bn.running_mean
bconv3.bn.running_var xnor.4.bn.running_var
bconv3.conv.weight xnor.4.conv.weight
bconv3.conv.bias xnor.4.conv.bias
bconv4.bn.weight xnor.6.bn.weight
bconv4.bn.bias xnor.6.bn.bias
bconv4.bn.running_mean xnor.6.bn.running_mean
bconv4.bn.running_var xnor.6.bn.running_var
bconv4.conv.weight xnor.6.conv.weight
bconv4.conv.bias xnor.6.conv.bias
bconv5.bn.weight xnor.7.bn.weight
bconv5.bn.bias xnor.7.bn.bias
bconv5.bn.running_mean xnor.7.bn.running_mean
bconv5.bn.running_var xnor.7.bn.running_var
bconv5.conv.weight xnor.7.conv.weight
bc

In [3]:
correct = 0
with torch.no_grad():
    for image, target in testloader:
        image, target = image.to(device), target.to(device)

        output = model(image)
        #test_loss += criterion(output, target).data.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum().cpu()
bin_op.restore()
acc = 100. * float(correct) / float(len(testloader.dataset))
print(acc)

86.28
